https://dumps.wikimedia.org/fawiki/20230601/

https://github.com/CyberZHG/wiki-dump-reader

https://github.com/Ramaseshanr/IITMDS/blob/main/NLP_Workshop_IITMDS.ipynb

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.15.1/
shasum -a 512 -c elasticsearch-oss-7.15.1-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

In [ ]:
# !gdown https://dumps.wikimedia.org/fawiki/20230601/fawiki-20230601-pages-articles-multistream.xml.bz2
!gdown https://dumps.wikimedia.org/fawiki/20230601/fawiki-20230601-pages-articles.xml.bz2

Downloading...
From: https://dumps.wikimedia.org/fawiki/20230601/fawiki-20230601-pages-articles.xml.bz2
To: /content/fawiki-20230601-pages-articles.xml.bz2
100% 1.05G/1.05G [03:37<00:00, 4.84MB/s]


In [ ]:
!bunzip2 fawiki-20230601-pages-articles.xml.bz2

In [ ]:
!pip install wiki-dump-reader
!pip install elasticsearch

  Preparing metadata (setup.py) ... done
  Created wheel for wiki-dump-reader: filename=wiki_dump_reader-0.0.4-py3-none-any.whl size=3983 sha256=036c047366932786075119c0d7e7c1d5fdf29bf20846c0769ca45bb7c9880f1d
  Stored in directory: /root/.cache/pip/wheels/78/81/3d/463b7f906f65d3e9e43db8446ebc5fb719bf1777a40b411cd2
Successfully built wiki-dump-reader
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.5/395.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [ ]:
from elasticsearch import Elasticsearch

cloud_id = "My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRiNTk1ZDliNGM4MjA0NzA2YmU0NDVmNWQ4MDc4YWE0YiRkNGZlMmFjNzJkMWU0NWY1YjRkMmZiODNlNTU4NmVjNg=="

# Create Elasticsearch client instance
client = Elasticsearch(cloud_id=cloud_id, basic_auth=('elastic', 'ujS3qg86qB7fy5WuqqlpX3CL'))

# Now you can perform actions using the client
response = client.search(index="your-index", body={"query": {"match_all": {}}})

In [ ]:
from elasticsearch import Elasticsearch

# Create an instance of Elasticsearch client
es = Elasticsearch(cloud_id=cloud_id, basic_auth=('lyrdaq', '37771234'))

# Define the index settings and mappings
index_name = "wiki"
settings = {
    "settings": {
        "number_of_shards": 5,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "text": {"type": "text"},
        }
    }
}

# Create the index with the specified settings and mappings
response = es.indices.create(index=index_name, body=settings)

# Print the response
print(response)


<ipython-input-6-f880b89863d3>:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.indices.create(index=index_name, body=settings)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'health_items'}


In [ ]:
from wiki_dump_reader import Cleaner,iterate

cleaner = Cleaner()

file_name='fawiki-20230601-pages-articles.xml'
page_count = 0

with open(f'{file_name}.cleaned.txt', 'w', encoding='utf-8') as output:
  for title, text in iterate(file_name):
    text = cleaner.clean_text(text)
    cleaned_text, links = cleaner.build_links(text)
    #print(f"{title} -> {cleaned_text}")

    output.write(title + '\n' + cleaned_text + '\n')

    page_count += 1
    if page_count % 50 == 0:
        print('Pages dumped = ', page_count)
    # For demo purposes, the execution is stopped after 5 pages
    # if page_count > 100:
    #     break

In [ ]:
!pip install elasticsearch

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Set up Spark session
spark = SparkSession.builder \
    .appName("WikipediaXMLParsing") \
    .getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from wiki_dump_reader import Cleaner, iterate

spark = SparkSession.builder.appName("WikipediaXMLParsing").getOrCreate()

file_name = 'fawiki-20230601-pages-articles.xml'
cleaner = Cleaner()

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(StringType())
def clean_text(text):
    return cleaner.clean_text(text)


wikipedia_df = spark.read.format("xml") \
    .option("rowTag", "page") \
    .load(file_name) \
    .withColumn("cleaned_text", clean_text(col("revision.text._VALUE"))) \
    .select("title", "cleaned_text")


wikipedia_df.show()

wikipedia_df.write \
    .format("org.elasticsearch.spark.sql") \
    .mode("append") \
    .option("es.resource", "wiki/doc") \
    .option("es.nodes", "localhost") \
    .option("es.port", "9200") \
    .save()


In [ ]:
%reset -f

In [ ]:
from wiki_dump_reader import Cleaner,iterate
from elasticsearch.helpers import bulk

cleaner = Cleaner()

file_name='fawiki-20230601-pages-articles.xml'
page_count = 0

batch_data = []

for title, text in iterate(file_name):
  text = cleaner.clean_text(text)
  cleaned_text, links = cleaner.build_links(text)
  #print(f"{title} -> {cleaned_text}")
  batch_data.append({"_index": "wiki", "_source":{"title": title, "text": cleaned_text}})
  page_count += 1
  if page_count % 100000 == 0:
      bulk(es, batch_data)
      batch_data = []
      print('Pages dumped = ', page_count)

bulk(es, batch_data)

In [ ]:
file_path = "illness.txt"

with open(file_path, "r") as file:
    illness = [line.strip() for line in file]

file_path = "medicines.txt"

with open(file_path, "r") as file:
    medicines = [line.strip() for line in file]

file_path = "signs.txt"

with open(file_path, "r") as file:
    signs = [line.strip() for line in file]

In [ ]:
medicines

In [ ]:
from elasticsearch import Elasticsearch

cloud_id = "wikipedia:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRjZGI0ZjYwMjI1ZTU0NjgyOGRiZDBlNWJhZmUxYWY1OSRlNzJhNjg4ZDU3MjI0YjM3OGVmZDlkYTFlM2M2MWFhMw=="

# Create Elasticsearch client instance
client = Elasticsearch(cloud_id=cloud_id, basic_auth=('lyrdaq', '37771234'))

In [ ]:
with open('graph2.txt', 'w', encoding='utf-8') as output:
  for illnes in illness:
    for sign in signs:
      query = {
        "query": {
          "bool": {
            "must": [
              {
                "term": {
                  "text": illnes
                }
              },
              {
                "term": {
                  "text": sign
                }
              }
            ]
          }
        }
      }
      result = client.count(index="wiki", body=query)
      count = str(result["count"])
      output.write(illnes + "," + sign + "," + count + '\n')
      print(illnes, sign, count)


In [ ]:
import pandas as pd

# Read the text file as a DataFrame
df = pd.read_csv('graph.txt', sep=',', header=None)

# Rename the columns
df.columns = ['medicine', 'sign', 'count']

# Sort the DataFrame by the "Number" column in descending order
df_sorted = df.sort_values(by='count', ascending=False)

# Display the sorted DataFrame
print(df_sorted.head())
df_sorted.to_csv('graph_signs_medicines.csv', index=False, encoding='utf-8-sig')

     medicine  sign  count
9782     شوید   درد     75
9801     شوید   ضعف     67
3867     پیاز   درد     58
8599     سبزی   درد     39
3858     پیاز  تورم     33


In [ ]:
df = pd.read_csv('graph_medicine_ilness.csv')
df.head()
df.to_csv("graph_signs_medicines2.csv", encoding='utf-8', index=False)

In [ ]:
text_input = '''سلام وقت بخیر.حدود ۳ ماه پیش کرونای خفیف گرفتم ولی تو اون مدت خیلی استرس داشتم بعد از پایان قرنطینه یک روز که بیرون بودم بعد از برگشت به خانه لرزش شدید پا گرفتم به همراه گیجی سر و الان حدود چند ماهی هست که حالت گیج و منگی بهمراه تپش قلب دارم به چند پزشک مراجعه کردم به اتفاق گفتند که دچار اضطراب شدم چند روزی هم دست و پاهام و سرم حالت گز گز و بی حسی پیدا کرده البته فقط چند روز هست که از قرص آرامبخش استفاده میکنم چون حس خوبی به قرص نداشتم ممنون میشم که نظرتون رو بگید'''

def extract_words(text_input, word_list):
    extracted_words = []
    words = text_input.split()

    for word in words:
        if word in word_list:
            extracted_words.append(word)

    return extracted_words

med = extract_words(text_input, illness)
print(med)

In [ ]:
!pip install hazm

In [ ]:
from hazm import *

stemmer = Stemmer()
stemmer.stem("تپش")

'تپ'

In [ ]:
from hazm import *

text_input = '''سلام وقت بخیر.حدود ۳ ماه پیش کرونای خفیف گرفتم ولی تو اون مدت خیلی استرس داشتم بعد از پایان قرنطینه یک روز که بیرون بودم بعد از برگشت به خانه لرزش شدید پا گرفتم به همراه گیجی سر و الان حدود چند ماهی هست که حالت گیج و منگی بهمراه تپش قلب دارم به چند پزشک مراجعه کردم به اتفاق گفتند که دچار اضطراب شدم چند روزی هم دست و پاهام و سرم حالت گز گز و بی حسی پیدا کرده البته فقط چند روز هست که از قرص آرامبخش استفاده میکنم چون حس خوبی به قرص نداشتم ممنون میشم که نظرتون رو بگید'''

def extract_words(text_input, word_list):
    extracted_words = []
    stemmer = Stemmer()
    words = [stemmer.stem(word) for word in word_tokenize(text_input)]

    for word in words:
        if word in word_list:
            extracted_words.append(word)

    return extracted_words

med = extract_words(text_input, signs)
print(med)

NameError: ignored

In [ ]:
file_path = "illness.txt"

with open(file_path, "r") as file:
    illness = [line.strip() for line in file]

file_path = "medicines.txt"

with open(file_path, "r") as file:
    medicines = [line.strip() for line in file]

file_path = "signs.txt"

with open(file_path, "r") as file:
    signs = [line.strip() for line in file]

In [ ]:
from hazm import *

text_input="سردرد"
stemmer = Stemmer()

def extract_ngrams(text_input, n):
    ngrams = []
    words = word_tokenize(text_input)
    if len(words) >= n:
        for i in range(len(words) - n + 1):
            ngram = tuple(words[i:i+n])
            ngrams.append(ngram)
    return ngrams


def filter_phrases(text_input, phrase_list, n):
    matched_phrases = []
    ngrams = extract_ngrams(text_input, n)

    for ngram in ngrams:
        phrase = " ".join(ngram)
        stem_phrase = " ".join([stemmer.stem(gram) for gram in ngram])
        phrase_ws = "".join(ngram)
        stem_phrase_ws = "".join([stemmer.stem(gram) for gram in ngram])

        if stem_phrase in phrase_list:
            matched_phrases.append(stem_phrase)
        if phrase in phrase_list:
            matched_phrases.append(phrase)

        if stem_phrase_ws in phrase_list:
            matched_phrases.append(stem_phrase_ws)
        if phrase_ws in phrase_list:
            matched_phrases.append(phrase_ws)

        for gram in ngram:
            if gram in phrase_list:
                matched_phrases.append(gram)

            stem_gram = stemmer.stem(gram)
            if stem_gram in phrase_list:
                matched_phrases.append(stem_gram)

    matched_phrases = list(set(matched_phrases))
    return matched_phrases

matched_signs = filter_phrases(text_input, signs, 2)
matched_medicines = filter_phrases(text_input, medicines, 2)
matched_illness = filter_phrases(text_input, illness, 2)

matched_phrases = {"matched_signs": matched_signs,
                   "matched_medicines": matched_medicines,
                   "matched_illness":matched_illness}
matched_phrases

{'matched_signs': [], 'matched_medicines': [], 'matched_illness': []}